In [15]:
import torch
import torch.nn.functional as F 
import matplotlib as plt 
%matplotlib inline


In [28]:
words=open("names.txt",'r').read().splitlines()
chars=sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
vocab_size=len(itos)
vocab_size

27

In [17]:
import random   
random.seed(42) 
random.shuffle(words)

In [63]:
block_size=3 

def build_dataset(words):
    x,y=[],[]

    for w in words:
        context=[0]*block_size 
        for ch in w + '.':
            ix=stoi[ch]
            x.append(context)
            y.append(ix)
            context=context[1:]+[ix]
    x=torch.tensor(x)
    y=torch.tensor(y)
    print(x.shape,y.shape)
    return (x,y)

n1=int(len(words)*0.8)
n2=int(len(words)*0.9)
xtr,ytr=build_dataset(words[:n1])
xdev,ydev=build_dataset(words[n1:n2])
xtest,ytest=build_dataset(words[n2:])


torch.Size([182778, 3]) torch.Size([182778])
torch.Size([22633, 3]) torch.Size([22633])
torch.Size([22735, 3]) torch.Size([22735])


In [64]:
for i in range(6):
    print([(itos[ch.item()]) for ch in xdev[i]],itos[ydev[i].item()])
    

['.', '.', '.'] a
['.', '.', 'a'] m
['.', 'a', 'm'] r
['a', 'm', 'r'] o
['m', 'r', 'o'] m
['r', 'o', 'm'] .


In [ ]:
class Linear:
    def __init__(self,fan_in,fan_out,bias=True):
        self.weight=torch.randn((fan_in,fan_out))/fan_in**0.5
        self.bias=torch.randn((fan_out)) if bias else None
    def __call__(self,x):
        self.out=x@self.weight
        if self.bias is not None:
            self.out+=self.bias 
        return self.out 
    def parameters(self):
        return [self.weight]+([] if self.bias is None else [self.bias])

class BatchNorm1d:
    
    def __init__(self,dim,eps=1e-5,momentum=0.1):
        self.eps=eps 
        self.momentum=momentum 
        self.training=True 

        self.gamma=torch.ones(dim)
        self.beta=torch.zeros(dim) 

        self.running_mean=torch.zeros(dim)
        self.running_var=torch.ones(dim)

    def __call__(self,x):
        if self.training:
            xmean=x.mean(0,keepdim=True)
            xvar=x.var(0,keepdim=True)
        else:
            xmean=self.running_mean 
            xvar=self.running_var 
        xhat=(x-xmean)/torch.sqrt(xvar+self.eps)
        self.out=self.gamma*xhat+self.beta 
        if self.training:
            with torch.no_grad():
                self.running_mean=(1-self.momentum)*self.running_mean+self.momentum*xmean
                self.running_var=(1-self.momentum)*self.running_var+self.momentum*xvar 
            return self.out 
    def parameters(self):
        return [self.gamma,self.beta]
    
class Tanh:
    def __call__(self,x):
        self.out=torch.tanh(x)
        return self.out 
    def parameters(sefl):
        return []
    

n_embd=10 
n_hidden=100 
g=torch.Generator().manual_seed(2147483647)
c=torch.randn((vocab_size,n_embd), generator=g)

layers=[
    Linear(n_embd*block_size,n_hidden,bias=False), BatchNorm1d(n_hidden),Tanh(),
    Linear()
]

    